# 🏗️ Creating and Declaring New Classes - Semantic Objects Tutorial

This tutorial teaches you how to create your own semantic object classes, following the patterns used in the library. You'll learn to define entities, properties, and relationships just like the built-in `Space`, `Window`, and `Area` classes.

## What You'll Learn

1. **Understanding the `@semantic_object` decorator**
2. **Creating Entity classes** (like Space, Window)
3. **Creating Property classes** (like Area, Temperature)
4. **Working with inheritance hierarchies**
5. **Defining relationships between classes**
6. **Advanced patterns and best practices**

Let's dive in! 🚀

## Setup and Imports

In [1]:
# Core imports
from semantic_objects.core import *
from semantic_objects.s223 import *
from semantic_objects.s223.relations import *
from semantic_objects.qudt import quantitykinds, units
from semantic_objects.exporters import export_templates
from semantic_objects.build_model import BMotifSession
from dataclasses import field
from typing import Optional, List
from pprint import pprint

print("✅ Imports successful!")

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


✅ Imports successful!


## 1. 🎯 Understanding the `@semantic_object` Decorator

The `@semantic_object` decorator is the foundation of all semantic classes. It:

- Converts your class into a dataclass
- Handles inheritance of fields from parent classes
- Sets up default values for `_name` and `abstract` attributes
- Manages field metadata for relations and constraints

Let's see it in action:

In [2]:
# Basic semantic object
@semantic_object
class MyBasicEntity(Node):
    label = "My Basic Entity"
    comment = "A simple example entity"

# Let's inspect what the decorator did
print(f"Class name: {MyBasicEntity.__name__}")
print(f"Is dataclass: {hasattr(MyBasicEntity, '__dataclass_fields__')}")
print(f"Abstract: {MyBasicEntity.abstract}")
print(f"Label: {MyBasicEntity.label}")

# Create an instance
entity = MyBasicEntity()
print(f"Instance name: {entity._name}")

Class name: MyBasicEntity
Is dataclass: True
Abstract: False
Label: My Basic Entity
Instance name: MyBasicEntity_1


## 2. 🏢 Creating Entity Classes

Entity classes represent physical or logical objects in your domain. They inherit from `Node` and can have properties and relationships.

### Simple Entity with Properties

In [3]:
# First, let's create a power sensor using an existing quantity kind
@semantic_object
class PowerSensor(Node):
    label = "Power Sensor"
    comment = "A sensor that measures electrical power"
    
    # Required power reading (Power is in the default unit map)
    power: Power = required_field()
    
    # Optional location description
    location: Optional[str] = field(default=None, metadata={'relation': None})
    
    def __post_init__(self):
        """Convert raw power values to Power objects"""
        super().__post_init__()
        if not isinstance(self.power, Power):
            self.power = Power(self.power)

# Test our new sensor
sensor = PowerSensor(power=1500.0, location="Office 101")
print(f"Sensor: {sensor._name}")
print(f"Power: {sensor.power.value} {sensor.power.unit}")
print(f"Location: {sensor.location}")
print(f"Quantity Kind: {sensor.power.qk}")

Sensor: PowerSensor_1
Power: 1500.0 <class 'semantic_objects.qudt.units.KiloW'>
Location: Office 101
Quantity Kind: <class 'semantic_objects.qudt.quantitykinds.Power'>


### Entity with Multiple Properties

In [4]:
# Create additional property types using supported quantity kinds
@semantic_object
class PowerConsumption(QuantifiableObservableProperty):
    qk = quantitykinds.Power
    _semantic_type = QuantifiableObservableProperty

@semantic_object
class AirPressure(QuantifiableObservableProperty):
    qk = quantitykinds.Pressure
    _semantic_type = QuantifiableObservableProperty

# Multi-sensor entity
@semantic_object
class EnvironmentalStation(Node):
    label = "Environmental Station"
    comment = "A comprehensive environmental monitoring station"
    
    power_consumption: PowerConsumption = required_field()
    air_pressure: AirPressure = required_field()
    
    # Optional fields
    station_id: Optional[str] = optional_field(relation=label)
    
    def __post_init__(self):
        """Convert raw values to proper property types"""
        super().__post_init__()
        
        if not isinstance(self.power_consumption, PowerConsumption):
            self.power_consumption = PowerConsumption(self.power_consumption)
        if not isinstance(self.air_pressure, AirPressure):
            self.air_pressure = AirPressure(self.air_pressure)

# Create an environmental station
station = EnvironmentalStation(
    power_consumption=2500.0,  # kW (default SI unit)
    air_pressure=101325.0,     # Pa (default SI unit)
    station_id="ENV001"
)

print(f"Environmental Station: {station._name}")
print(f"Power Consumption: {station.power_consumption.value} {station.power_consumption.unit}")
print(f"Air Pressure: {station.air_pressure.value} {station.air_pressure.unit}")
print(f"Station ID: {station.station_id}")

TypeError: EnvironmentalStation.__init__() got an unexpected keyword argument 'station_id'

## 3. 📊 Creating Property Classes

Property classes represent measurable attributes. They typically inherit from `QuantifiableObservableProperty` and define a quantity kind.

### Basic Property Pattern

In [ ]:
# Property with fixed unit using existing supported types
@semantic_object
class Power_kW(QuantifiableObservableProperty):
    qk = quantitykinds.Power
    unit = units.KiloW  # Fixed to kilowatts
    _semantic_type = QuantifiableObservableProperty

# Test the properties
generic_power = Power(1500.0)  # Uses default unit from DEFAULT_UNIT_MAP
kw_power = Power_kW(1.5)       # Fixed to kilowatts

print(f"Generic power: {generic_power.value} {generic_power.unit}")
print(f"kW power: {kw_power.value} {kw_power.unit}")
print(f"Same quantity kind: {generic_power.qk == kw_power.qk}")

Generic power: 1500.0 <class 'semantic_objects.qudt.units.KiloW'>
kW power: 1.5 <class 'semantic_objects.qudt.units.KiloW'>
Same quantity kind: True


### Property with Aspects (Enumerations)

In [ ]:
# Import enumeration kinds
from semantic_objects.s223.enumerationkinds import Setpoint, Deadband, Occupancy

# Property that can have aspects (like setpoint, deadband)
# Let's use Area as an example since it's well supported
@semantic_object
class Area_SP(Area):
    """Area property that can have setpoint/deadband aspects"""
    _semantic_type = QuantifiableObservableProperty
    
    aspects: Optional[list] = field(
        default=None,
        init=False,
        metadata={
            'relation': hasAspect,
            'exact_values': [Setpoint, Deadband],
            'qualified': False
        }
    )

# Test the property with aspects
setpoint_area = Area_SP(100.0)
print(f"Area setpoint: {setpoint_area.value} {setpoint_area.unit}")
print(f"Available aspects: {Area_SP.__dataclass_fields__['aspects'].metadata['exact_values']}")

Area setpoint: 100.0 <class 'semantic_objects.qudt.units.M2'>
Available aspects: [<class 'semantic_objects.s223.enumerationkinds.Setpoint'>, <class 'semantic_objects.s223.enumerationkinds.Deadband'>]


## 4. 🔗 Working with Inheritance Hierarchies

Semantic objects support rich inheritance patterns. Let's create a hierarchy of HVAC equipment:

In [ ]:
# Base equipment class
@semantic_object
class Equipment(Node):
    label = "Equipment"
    comment = "Base class for all equipment"
    abstract = True  # This class won't be instantiated directly
    
    # Common properties for all equipment
    power: Power = required_field()

# HVAC equipment specialization
@semantic_object
class HVACEquipment(Equipment):
    label = "HVAC Equipment"
    comment = "Equipment used for heating, ventilation, and air conditioning"
    abstract = True

# Specific equipment types
@semantic_object
class AirHandlingUnit(HVACEquipment):
    label = "Air Handling Unit"
    comment = "Equipment that conditions and circulates air"
    
    # Additional properties specific to AHUs - using supported quantity kinds
    supply_pressure: AirPressure = required_field()
    return_pressure: AirPressure = required_field()
    
    def __post_init__(self):
        super().__post_init__()
        if not isinstance(self.power, Power):
            self.power = Power(self.power)
        if not isinstance(self.supply_pressure, AirPressure):
            self.supply_pressure = AirPressure(self.supply_pressure)
        if not isinstance(self.return_pressure, AirPressure):
            self.return_pressure = AirPressure(self.return_pressure)

@semantic_object
class Chiller(HVACEquipment):
    label = "Chiller"
    comment = "Equipment that removes heat from a liquid via a vapor-compression cycle"
    
    # Chiller-specific properties - using power consumption as proxy for efficiency
    cooling_power: PowerConsumption = required_field()
    operating_pressure: AirPressure = required_field()
    
    def __post_init__(self):
        super().__post_init__()
        if not isinstance(self.power, Power):
            self.power = Power(self.power)
        if not isinstance(self.cooling_power, PowerConsumption):
            self.cooling_power = PowerConsumption(self.cooling_power)
        if not isinstance(self.operating_pressure, AirPressure):
            self.operating_pressure = AirPressure(self.operating_pressure)

# Test the hierarchy
ahu = AirHandlingUnit(
    power=5000.0,
    supply_pressure=1200.0,  # Pa
    return_pressure=1000.0   # Pa
)

chiller = Chiller(
    power=150000.0,
    cooling_power=120000.0,
    operating_pressure=500000.0  # Pa
)

print(f"AHU: {ahu._name}")
print(f"  Power: {ahu.power.value} {ahu.power.unit}")
print(f"  Supply pressure: {ahu.supply_pressure.value} {ahu.supply_pressure.unit}")

print(f"\nChiller: {chiller._name}")
print(f"  Power: {chiller.power.value} {chiller.power.unit}")
print(f"  Cooling power: {chiller.cooling_power.value} {chiller.cooling_power.unit}")

# Show inheritance
print(f"\nInheritance check:")
print(f"AHU is Equipment: {isinstance(ahu, Equipment)}")
print(f"AHU is HVACEquipment: {isinstance(ahu, HVACEquipment)}")
print(f"Chiller is Equipment: {isinstance(chiller, Equipment)}")

AHU: AirHandlingUnit_1
  Power: 5000.0 <class 'semantic_objects.qudt.units.KiloW'>
  Supply pressure: 1200.0 <class 'semantic_objects.qudt.units.PA'>

Chiller: Chiller_1
  Power: 150000.0 <class 'semantic_objects.qudt.units.KiloW'>
  Cooling power: 120000.0 <class 'semantic_objects.qudt.units.KiloW'>

Inheritance check:
AHU is Equipment: True
AHU is HVACEquipment: True
Chiller is Equipment: True


## 5. 🔄 Defining Relationships Between Classes

You can define relationships between entities using inter-field relations and custom relation properties.

### Simple Relationships

In [ ]:
# Create a room that contains equipment
@semantic_object
class MechanicalRoom(Space):
    label = "Mechanical Room"
    comment = "A room that contains mechanical equipment"
    
    # Equipment in this room
    equipment: Optional[List[Equipment]] = field(
        default=None, 
        metadata={'relation': contains, 'qualified': False}
    )

# Create a system that connects multiple pieces of equipment
@semantic_object
class ChilledWaterSystem(Node):
    label = "Chilled Water System"
    comment = "A system that circulates chilled water"
    
    chiller: Chiller = required_field(relation=connectedFrom)
    air_handling_units: List[AirHandlingUnit] = field(
        default_factory=list,
        metadata={'relation': connectedTo, 'qualified': False}
    )

# Test relationships
mech_room = MechanicalRoom(area=50.0)
chw_system = ChilledWaterSystem(
    chiller=chiller,
    air_handling_units=[ahu]
)

print(f"Mechanical Room: {mech_room._name}")
print(f"Chilled Water System: {chw_system._name}")
print(f"  Chiller: {chw_system.chiller._name}")
print(f"  AHUs: {[ahu._name for ahu in chw_system.air_handling_units]}")

Mechanical Room: MechanicalRoom_1
Chilled Water System: ChilledWaterSystem_1
  Chiller: Chiller_1
  AHUs: ['AirHandlingUnit_1']


### Inter-Field Relations

In [ ]:
# Create a more complex relationship using inter-field relations
@semantic_object
class ZoneWithSensor(Space):
    label = "Zone with Sensor"
    comment = "A space that has a property"
    
    # The space itself
    zone: Space = required_field(relation=hasDomainSpace)
    
    # Temperature sensor in the space
    sensor: QuantifiableObservableProperty = required_field()
    
    # Define that the sensor is located in the zone
    _inter_field_relations = [
        inter_field_relation(
            source_field='zone',
            relation=contains,
            target_field='sensor',
            min=1,
            max=1
        )
    ]

# Create instances
office_space = Space(area=25.0)
office_space._name = "Office_205"

power_sensor = PowerSensor(power=1200.0, location="Office 205")

zone_with_sensor = ZoneWithSensor(
    area=25.0,  # This gets inherited from Space
    zone=office_space,
    sensor=power_sensor
)

print(f"Zone with Sensor: {zone_with_sensor._name}")
print(f"  Zone: {zone_with_sensor.zone._name}")
print(f"  Sensor: {zone_with_sensor.sensor._name}")
print(f"  Sensor reading: {zone_with_sensor.sensor.power.value} {zone_with_sensor.sensor.power.unit}")
print(f"  Inter-field relations: {len(zone_with_sensor._inter_field_relations)}")

Zone with Sensor: ZoneWithSensor_1
  Zone: Office_205
  Sensor: PowerSensor_2
  Sensor reading: 1200.0 <class 'semantic_objects.qudt.units.KiloW'>
  Inter-field relations: 1


## 8. 📤 Exporting and Using Your Classes

Now let's export templates and generate RDF for our custom classes:

In [ ]:
# Export templates for our custom classes
export_templates(EnvironmentalStation, 'custom_templates')
export_templates(AirHandlingUnit, 'custom_templates')

print("✅ Custom templates exported")

# Generate SPARQL queries
print("\n=== Generated SPARQL Queries ===")
env_query = EnvironmentalStation.get_sparql_query(ontology='s223')
print("EnvironmentalStation query:")
print(env_query[:200] + "..." if len(env_query) > 200 else env_query)

# Generate RDF class definitions
print("\n=== RDF Class Definition ===")
ahu_rdf = AirHandlingUnit.generate_rdf_class_definition(include_hierarchy=False)
print("AirHandlingUnit RDF (first 500 chars):")
print(ahu_rdf[:500] + "..." if len(ahu_rdf) > 500 else ahu_rdf)

AttributeError: 'str' object has no attribute '_name'

## 9. 🏗️ Building Models with Custom Classes

Let's create a complete model using our custom classes:

In [ ]:
# Create a BMotifSession for our custom model
custom_session = BMotifSession(ns='custom_building')

# Load templates for our custom classes
custom_session.load_class_templates(EnvironmentalStation)
custom_session.load_class_templates(AirHandlingUnit)
custom_session.load_class_templates(BuildingFloor)

print(f"Available templates: {list(custom_session.templates.keys())}")

# Create and evaluate our objects
objects_to_evaluate = [
    station,      # EnvironmentalStation
    ahu,          # AirHandlingUnit
    second_floor, # BuildingFloor with spaces
    power_meter   # SmartPowerMeter
]

for obj in objects_to_evaluate:
    try:
        custom_session.evaluate(obj)
        print(f"✅ Evaluated {obj._name} ({obj.__class__.__name__})")
    except Exception as e:
        print(f"❌ Failed to evaluate {obj._name}: {e}")

# Show generated RDF
print(f"\nGenerated {len(custom_session.graph)} RDF triples")
rdf_output = custom_session.graph.serialize(format='turtle')
print("\nSample RDF output:")
print(rdf_output[:800] + "..." if len(rdf_output) > 800 else rdf_output)

## 10. 📋 Best Practices Summary

### ✅ Do's

1. **Use the `@semantic_object` decorator** on all your semantic classes
2. **Inherit from appropriate base classes**:
   - `Node` for entities (physical/logical objects)
   - `QuantifiableObservableProperty` for measurable properties
   - `Predicate` for relationships
3. **Define clear labels and comments** for documentation
4. **Use `required_field()` for mandatory properties**
5. **Implement `__post_init__`** for automatic type conversion
6. **Set `abstract = True`** for base classes that shouldn't be instantiated
7. **Use factory methods** for complex object creation
8. **Define inter-field relations** for complex relationships

### ❌ Don'ts

1. **Don't forget the decorator** - your class won't work properly
2. **Don't mix relation types** - be consistent with your relationship patterns
3. **Don't create circular dependencies** in your class hierarchy
4. **Don't ignore type hints** - they're crucial for proper functioning
5. **Don't hardcode values** that should be configurable

### 🎯 Key Patterns

```python
# Basic entity pattern
@semantic_object
class MyEntity(Node):
    label = "My Entity"
    property: MyProperty = required_field()

# Property pattern
@semantic_object
class MyProperty(QuantifiableObservableProperty):
    qk = quantitykinds.SomeQuantityKind
    _semantic_type = QuantifiableObservableProperty

# Inheritance pattern
@semantic_object
class SpecializedEntity(MyEntity):
    additional_property: AnotherProperty = required_field()
```

## 🎉 Congratulations!

You now know how to create your own semantic object classes! You can:

- Define entities and properties
- Create inheritance hierarchies
- Set up relationships between classes
- Export templates and generate RDF
- Build complete semantic models

Start building your own domain-specific semantic objects! 🚀